In [17]:
import os
import subprocess
import sys
import ribasim
import ribasim.nodes
import toml
import tomli
import numpy as np
import pandas as pd
import pathlib
import tqdm.auto as tqdm
import warnings
import datetime
from pathlib import Path
import logging

from shapely.geometry import Point

#load ribasim_nl
module_path = Path.cwd() / '../../ribasim_nl/'
sys.path.append(str(module_path))
from ribasim_nl import CloudStorage

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

%reload_ext autoreload
%autoreload 2
warnings.filterwarnings('ignore')
    
import peilbeheerst_model.ribasim_parametrization as ribasim_param
from peilbeheerst_model.ribasim_feedback_processor import RibasimFeedbackProcessor

from peilbeheerst_model.controle_output import *
from ribasim import Solver

from ribasim import Allocation, Model, Node
from ribasim.nodes import (
    basin,
    discrete_control,
    flow_boundary,
    fractional_flow,
    level_boundary,
    level_demand,
    linear_resistance,
    manning_resistance,
    outlet,
    pid_control,
    pump,
    tabulated_rating_curve,
    user_demand,
)

## Define variables and model

#### Set Config

In [18]:
# Set paths
waterschap = "HollandseDelta"
versie = "2024_6_1"

work_dir = pathlib.Path(r"../../../../../Ribasim_updated_models", waterschap)
ribasim_gpkg = work_dir.joinpath("database.gpkg")
path_ribasim_toml = work_dir.joinpath("ribasim.toml")
output_dir = work_dir.joinpath("results")

# Basin area percentage
regular_percentage = 10
boezem_percentage = 90
unknown_streefpeil = 0.00012345 #we need a streefpeil to create the profiles, Q(h)-relations, and af- and aanslag peil for pumps

# Forcing settings
start_time = "2024-01-01"
timestep_size = 'd'
timesteps = 2


## Process the feedback form

In [19]:
name = "Ron Bruijns (HKV)"

feedback_excel = pathlib.Path(r"../../../../../Ribasim_feedback/V1_formulieren/feedback_formulier_" + waterschap + ".xlsx")
feedback_excel_processed = r"../../../../..//Ribasim_feedback/V1_formulieren_verwerkt/feedback_formulier_" + waterschap + "_JA_processed.xlsx"

ribasim_toml = pathlib.Path(r"../../../../../Ribasim_base_models", waterschap + '_boezemmodel_' + versie, 'ribasim.toml')
output_folder = work_dir #r"../../../../../Ribasim_updated_models/AmstelGooienVecht"

processor = RibasimFeedbackProcessor(name, waterschap, versie, feedback_excel, ribasim_toml, output_folder, feedback_excel_processed)
processor.run()

Removed node (and edges) with Node Type: tabulated_rating_curve and Node ID: 1619
Removed node (and edges) with Node Type: tabulated_rating_curve and Node ID: 1619
Removed node (and edges) with Node Type: flow_boundary and Node ID: 1620
Removed node (and edges) with Node Type: flow_boundary and Node ID: 1620
Removed node (and edges) with Node Type: flow_boundary and Node ID: 1522
Removed node (and edges) with Node Type: flow_boundary and Node ID: 1522
Removed node (and edges) with Node Type: terminal and Node ID: 2634
Removed node (and edges) with Node Type: tabulated_rating_curve and Node ID: 1786
Removed node (and edges) with Node Type: tabulated_rating_curve and Node ID: 1786
675 2682
675 170
135 703
2610 361
Removed node (and edges) with Node Type: tabulated_rating_curve and Node ID: 1086
Removed node (and edges) with Node Type: tabulated_rating_curve and Node ID: 1086
Removed node (and edges) with Node Type: flow_boundary and Node ID: 1087
Removed node (and edges) with Node Type: 

#### Load model

In [20]:
# Load Ribasim model
with warnings.catch_warnings():
    warnings.simplefilter(action='ignore', category=FutureWarning)
    ribasim_model = ribasim.Model(filepath=path_ribasim_toml)

## Model specific tweaks

In [21]:
assert not pd.isnull(ribasim_model.basin.area.df.meta_streefpeil).any()

ribasim_model.basin.area.df.loc[ribasim_model.basin.area.df['meta_streefpeil'] == 'Onbekend streefpeil', 'meta_streefpeil'] = unknown_streefpeil

ribasim_model.basin.area.df.loc[ribasim_model.basin.area.df['meta_streefpeil'] == 9.999, 'meta_streefpeil'] = unknown_streefpeil

### Remove the TRC's without geometries

In [22]:
#sometimes the geometries are POINT EMPTY
ribasim_model.tabulated_rating_curve.node.df = ribasim_model.tabulated_rating_curve.node.df[~ribasim_model.tabulated_rating_curve.node.df['geometry'].is_empty]

#sometimes the geometries are None
ribasim_model.tabulated_rating_curve.node.df.dropna(subset='geometry', inplace = True)

In [23]:
new_node_id = max(ribasim_model.edge.df.from_node_id.max(), ribasim_model.edge.df.to_node_id.max()) + 1

### First manual adjustment: insert Terminal with TRC 
ribasim_model.terminal.add(Node(new_node_id, Point(98200,426447)))

new_node_id += 1
ribasim_model.tabulated_rating_curve.add(
    Node(new_node_id, Point(98200,426447)),
    [tabulated_rating_curve.Static(level=[0.1, 1.1], flow_rate=[0.0, 10])])

ribasim_model.edge.add(ribasim_model.basin[389], ribasim_model.tabulated_rating_curve[new_node_id])

ribasim_model.edge.add(ribasim_model.tabulated_rating_curve[new_node_id], ribasim_model.terminal[new_node_id-1])


# Parameterization

## Nodes

### Basin (characteristics)

In [24]:
# Define the initial state of each basin. Is set equal to the streefpeil
# ribasim_param.set_initial_basin_state(ribasim_model) #the initial states of the basins are by default already set to the streefpeil!

# Insert standard profiles to each basin. These are [depth_profiles] meter deep, defined from the streefpeil
ribasim_param.insert_standard_profile(ribasim_model, 
                                      regular_percentage = regular_percentage, 
                                      boezem_percentage = boezem_percentage,
                                      depth_profile = 2)

In [ ]:
# Add storage basins
# model_name = 'AmstelGooienVecht_StorageBasins'
# node_ids = [1, 2, 3]  # Specify node IDs to process or include_hoofdwater Boolean
processor = AddStorageBasin(ribasim_toml, model_name, output_folder, include_hoofdwater=True, log=True, node_ids=None)
ribasim_model = processor.run()

# Check basin area
ribasim_param.validate_basin_area(ribasim_model)

### Basin (forcing)

In [25]:
# Set static forcing
forcing_dict = {'precipitation':         ribasim_param.convert_mm_day_to_m_sec(5),
                'potential_evaporation': ribasim_param.convert_mm_day_to_m_sec(0),
                'drainage':              ribasim_param.convert_mm_day_to_m_sec(0),
                'infiltration':          ribasim_param.convert_mm_day_to_m_sec(0),
                'urban_runoff':          ribasim_param.convert_mm_day_to_m_sec(0),
                }

ribasim_param.set_static_forcing(timesteps, timestep_size, start_time, forcing_dict, ribasim_model)


### Pumps

In [26]:
# Set pump capacity for each pump
ribasim_model.pump.static.df['flow_rate'] = 0.16667 # 10 kuub per minuut

### FlowBoundary

In [27]:
# Set FlowBoundary rate for each pump
ribasim_model.flow_boundary.static.df['flow_rate'] = 0 #

### Add Discrete Control

In [28]:
# Add discrete control nodes and control edges
ribasim_param.add_discrete_control_nodes(ribasim_model)

Model bevat al discrete controls! Dan voegen we geen extra controls toe...


### TabulatedRatingCurve

In [29]:
# First, set the Q(h)-relationship to a relationship which is at each TRC node the same
ribasim_param.set_tabulated_rating_curves(ribasim_model, level_increase=1.0, flow_rate=1)

# The water level in the boezem is regulated heavily. There is a smaller tolerance for an increase in water level. 
# Hence, the TRC's which discharge water from the boezem to a Terminal should have a higher capacity.
ribasim_param.set_tabulated_rating_curves_boundaries(ribasim_model, level_increase=0.10, flow_rate=40)


# Set numerical settings

In [30]:
# Write model output
ribasim_param.index_reset(ribasim_model)
ribasim_model.starttime = datetime.datetime(2024, 1, 1)
ribasim_model.endtime = datetime.datetime(2024, 1, 5)
ribasim_model.solver.saveat = 3600
ribasim_param.write_ribasim_model_Zdrive(ribasim_model, 
                                         path_ribasim_toml)

## Run Model

## Iterate over tabulated rating curves

In [31]:
try:
    ribasim_param.iterate_TRC(ribasim_param = ribasim_param, 
                          allowed_tolerance = 0.02, 
                          max_iter = 1, 
                          expected_difference = 0.1, 
                          max_adjustment = 0.25, 
                          cmd = ['ribasim', path_ribasim_toml], 
                          output_dir = output_dir, 
                          path_ribasim_toml = path_ribasim_toml)

except Exception as e:
    logging.error("The model was not able to run. Log file:")
    log_file_path = os.path.join(output_dir, 'ribasim.log') 
    try:
        with open(log_file_path, 'r') as log_file:
            log_content = log_file.read()
            print(log_content)
    except Exception as log_exception:
        logging.error(f"Could not read the log file: {log_exception}")

Simulating init:   0%|          | 0/100 [00:00<?, ?it/s]

ERROR:root:The model was not able to run. Log file:


┌ Info: Starting a Ribasim simulation.
│   cli.ribasim_version = 2024.8.0
│   starttime = 2024-01-01T00:00:00
│   endtime = 2024-01-05T00:00:00
└ @ Ribasim /opt/teamcityagent/work/ecd2b8f9b25b1609/ribasim/core/src/main.jl:39



In [16]:
stop

NameError: name 'stop' is not defined

# Write model

In [32]:
# control_dict = Control(work_dir = work_dir).run_all()
ribasim_param.write_ribasim_model_GoodCloud(ribasim_model = ribasim_model, 
                                            path_ribasim_toml = path_ribasim_toml,
                                            waterschap = waterschap, 
                                            modeltype = 'boezemmodel',
                                            include_results = True)


INFO:pyogrio._io:Created 3,885 records
INFO:pyogrio._io:Created 3,085 records
INFO:pyogrio._io:Created 794 records
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): deltares.thegood.cloud:443
DEBUG:urllib3.connectionpool:https://deltares.thegood.cloud:443 "GET /remote.php/dav/files/nhi_api/Ribasim%20modeldata HTTP/1.1" 200 None
INFO:ribasim_nl.cloud:valid credentials
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): deltares.thegood.cloud:443
DEBUG:urllib3.connectionpool:https://deltares.thegood.cloud:443 "PROPFIND /remote.php/dav/files/nhi_api/Ribasim%20modeldata/HollandseDelta/modellen HTTP/1.1" 207 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): deltares.thegood.cloud:443
DEBUG:urllib3.connectionpool:https://deltares.thegood.cloud:443 "MKCOL /remote.php/dav/files/nhi_api/Ribasim%20modeldata/HollandseDelta/modellen/HollandseDelta_boezemmodel_2024_6_4 HTTP/1.1" 201 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): deltar

The model of waterboard HollandseDelta has been uploaded to the goodcloud in the directory of boezemmodel!


## Open Output

In [ ]:
df_basin = pd.read_feather(os.path.join(output_dir, 'basin.arrow'))
df_basin